# Generate and Perform Tiny Performances from the MDRNN

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
# little path hack to get robojam from one directory up in the filesystem.
from context import * # imports robojam
# import robojam # alternatively do this.

### Imports and Utility Functions

Functions are provided to generate performances of a certain length, plot, and perform them.

In [2]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# # tiny performance loader
# import tiny_performance_loader
# loader = tiny_performance_loader.TinyPerformanceLoader(verbose=False)

## Plotting Methods

Mainly using "plot_and_perform" method to generate 2D and 3D plots.

In [4]:
input_colour = 'darkblue'
gen_colour = 'firebrick'
plt.style.use('seaborn-talk')

def plot_2D(perf_df, name="foo"):
    """Plot in 2D"""
    ## Plot the performance
    swipes = divide_performance_into_swipes(perf_df)
    plt.figure(figsize=(8, 8))
    for swipe in swipes:
        p = plt.plot(swipe.x, swipe.y, 'o-')
        plt.setp(p, color=gen_colour, linewidth=5.0)
    plt.ylim(1.0,0)
    plt.xlim(0,1.0)
    plt.xticks([])
    plt.yticks([])
    #plt.axis('off')
    plt.savefig(name+".png", bbox_inches='tight')
    plt.close()

def plot_double_2d(perf1, perf2, name="foo"):
    """Plot two performances in 2D"""
    plt.figure(figsize=(8, 8))
    swipes = divide_performance_into_swipes(perf1)
    for swipe in swipes:
        p = plt.plot(swipe.x, swipe.y, 'o-')
        plt.setp(p, color=input_colour, linewidth=5.0)
    swipes = divide_performance_into_swipes(perf2)
    for swipe in swipes:
        p = plt.plot(swipe.x, swipe.y, 'o-')
        plt.setp(p, color=gen_colour, linewidth=5.0)
    plt.ylim(1.0,0)
    plt.xlim(0,1.0)
    plt.xticks([])
    plt.yticks([])
    plt.savefig(name+".png", bbox_inches='tight')
    plt.close()

def plot_3D(perf_df):
    """Plot in 3D"""
    ## Plot in 3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(perf_df.time, perf_df.x, perf_df.y, '.r-')
    ax.set_ylim(0,1.0)
    ax.set_zlim(1.0,0)
    #plt.axis('off')
    plt.show()

def plot_and_perform(perf, perform=True):
    """Plot in 3D and perform over OSC."""
    perf = perf.T
    perf_df = pd.DataFrame({'x':perf[0], 'y':perf[1], 't':perf[2]})
    perf_df['time'] = perf_df.t.cumsum()
    
    ## Plot the performance
    plt.figure(figsize=(8, 8))
    p = plt.plot(perf_df.x, perf_df.y, 'or-')
    plt.ylim(1.0,0)
    plt.xlim(0,1.0)
    #plt.axis('off')
    plt.setp(p, color='r', linewidth=5.0)
    plt.show()
    
    ## Plot in 3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(perf_df.time, perf_df.x, perf_df.y, '.r-')
    ax.set_ylim(0,1.0)
    ax.set_zlim(1.0,0)
    #plt.axis('off')
    plt.show()
    
    ## Perform
    if perform:
        playPerformance_XY_only(perf_df)
        
def plot_and_perform_sequentially(perf1, perf2, perform=True):
    total = np.append(perf1, perf2, axis=0)
    total = total.T
    perf1 = perf1.T
    perf2 = perf2.T
    perf1_df = pd.DataFrame({'x':perf1[0], 'y':perf1[1], 't':perf1[2]})
    perf2_df = pd.DataFrame({'x':perf2[0], 'y':perf2[1], 't':perf2[2]})
    total_df = pd.DataFrame({'x':total[0], 'y':total[1], 't':total[2]})
    perf1_df['time'] = perf1_df.t.cumsum()
    total_perf1_time = perf1_df.t.sum()
    perf2_df['time'] = perf2_df.t.cumsum() + total_perf1_time
    total_df['time'] = total_df.t.cumsum()
    ## Plot the performances
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(perf1_df.time, perf1_df.x, perf1_df.y, '.b-')
    ax.plot(perf2_df.time, perf2_df.x, perf2_df.y, '.r-')
    plt.show()
    if perform:
        playPerformance_XY_only(total_df)
        
def divide_performance_into_swipes(perf_df):
    """Divides a performance into a sequence of swipe dataframes."""
    # print("Total Performance Touches:\n",len(perf_df))
    touch_starts = perf_df[perf_df.moving == 0].index
    performance_swipes = []
    # print("Number of swipe starts:", len(touch_starts))
    remainder = perf_df
    for att in touch_starts:
        swipe = remainder.iloc[remainder.index < att]
        performance_swipes.append(swipe)
        remainder = remainder.iloc[remainder.index >= att]
    performance_swipes.append(remainder)
    # print("Number of Swipes",len(performance_swipes))
    # print("Total touches in swipes:", sum([len(x) for x in performance_swipes]))
    return performance_swipes

## Generate and play a performance

Performances are generated using the `generate_random_tiny_performance` method which is set to produce performances up to 5 seconds. The LSTM state and first touch can optionally be kept from the last evaluation or re-initialised.

This block can be run multiple times to generate more performances.

In [7]:
# Hyperparameters:
HIDDEN_UNITS = 512
LAYERS = 3
MIXES = 16
net = robojam.MixtureRNN(mode=robojam.NET_MODE_RUN, n_hidden_units=HIDDEN_UNITS, n_mixtures=MIXES, batch_size=1, sequence_length=1, n_layers=LAYERS)
# Setup synth for performance
setSynth(instrument = "chirp")
# chooseRandomSynth()

model_file = "../models/mdrnn-2d-1d-3layers-512units-16mixtures"

TEMPERATURE = 1.00

for i in range(96):
    name = "image_evaluation/uncond/touchperf-" + str(i)
    net.state = None # reset state if needed.
    perf = mixture_rnn.generate_random_tiny_performance(net, np.array([0.5, 0.5, 0.1]), time_limit=5.0, temp=TEMPERATURE, model_file=model_file)
    perf_df = mixture_rnn.perf_array_to_df(perf)
    plot_2D(perf_df, name=name)

INFO:tensorflow:Loading Running Operations
INFO:tensorflow:done initialising: mdrnn-2d-1d-3layers-512units-16mixtures vars: 5329040


NameError: name 'setSynth' is not defined

# Condition and Generate

Conditions the MDRNN on a random MicroJam performance, then generates an extra 5 seconds.

In [5]:
HIDDEN_UNITS = 512
LAYERS = 3
MIXES = 16
net = mixture_rnn.MixtureRNN(mode=mixture_rnn.NET_MODE_RUN, n_hidden_units=HIDDEN_UNITS, n_mixtures=MIXES, batch_size=1, sequence_length=1, n_layers=LAYERS)

model_file = "models/mdrnn-2d-1d-3layers-512units-16mixtures"

TEMPERATURE = 1.00
NUMBER = 96
input_perf_dfs = loader.sample_without_replacement(n=NUMBER)

for i, in_df in enumerate(input_perf_dfs):
    title = "image_evaluation/cond/condperf-" + str(i)
    in_array = mixture_rnn.perf_df_to_array(in_df)
    in_time = in_array.T[2].sum()
    print("In Time:", in_time)
    output_perf = mixture_rnn.condition_and_generate(net, in_array, time_limit=5.0, temp=TEMPERATURE, model_file=model_file)
    out_df = mixture_rnn.perf_array_to_df(output_perf)
    print("Out Time:", output_perf.T[2].sum())
    plot_double_2d(in_df, out_df, name=title)
    # plot_and_perform_sequentially(input_perf, output_perf, perform=True)

INFO:tensorflow:Loading Running Operations
INFO:tensorflow:done initialising: mdrnn-2d-1d-3layers-512units-16mixtures vars: 5329040
In Time: 0.914228
INFO:tensorflow:Restoring parameters from models/mdrnn-2d-1d-3layers-512units-16mixtures
Out Time: 5.04556432564
In Time: 3.485432
INFO:tensorflow:Restoring parameters from models/mdrnn-2d-1d-3layers-512units-16mixtures
Out Time: 9.196583032
In Time: 3.697068
INFO:tensorflow:Restoring parameters from models/mdrnn-2d-1d-3layers-512units-16mixtures
Out Time: 5.13981669652
In Time: 4.619267
INFO:tensorflow:Restoring parameters from models/mdrnn-2d-1d-3layers-512units-16mixtures
Out Time: 9.15391202418
In Time: 4.934818
INFO:tensorflow:Restoring parameters from models/mdrnn-2d-1d-3layers-512units-16mixtures
Out Time: 7.14371518869
In Time: 25.649282
INFO:tensorflow:Restoring parameters from models/mdrnn-2d-1d-3layers-512units-16mixtures
Out Time: 5.02560523606
In Time: 5.0134
INFO:tensorflow:Restoring parameters from models/mdrnn-2d-1d-3layer